# Getting the info that the client needs for D3.js

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import sys,pickle
sys.path.append('../')
from lib import modify_character

from lib import builders

with (open('world_4.p', 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
world

In [3]:
WORLD = builders.world
WORLD

<module 'lib.builders.world' from '../lib/builders/world.py'>

In [4]:
WORLD.get_area_data(world)

In [16]:
def chordKey(coord):
    '''
    takes a key [1,1], returns key "1,1"
    '''
    key = ":".join([str(i) for i in coord])
    return key

def get_features_or_NA(world,coord):
    try:
        l = world.df_features.loc[chordKey([coord[0],coord[1]])].to_dict(),
    except: 
        l = {"area is not on map"}
    return l

def get_area_data(self,world):
    """
    note: requires Character
    """
    mapData = {}
    coord = world.Character.location
    key = world.Character.get_location_key()
    l = world.df_features.loc[key]
    worldDim = {world}
    mapData = {'area':world.df_features.loc[key],
              'NArea':get_features_or_NA(world,[coord[0],coord[1]-1]),
              'SArea':get_features_or_NA(world,[coord[0],coord[1]+1]),
              'EArea':get_features_or_NA(world,[coord[0]+1,coord[1]]),
              'WArea':get_features_or_NA(world,[coord[0]-1,coord[1]])}
    return mapData
    
get_area_data("foo",world)

{'area': y                        30
 rainfall                 15
 x                        14
 key                   14:30
 elevation                 2
 terrain                town
 feature           Hellplace
 nation number             0
 nation           Flamehaven
 z                      City
 Name: 14:30, dtype: object,
 'NArea': ({'y': 29,
   'rainfall': 9.0,
   'x': 14,
   'key': '14:29',
   'elevation': 0.0,
   'terrain': 'ocean',
   'feature': nan,
   'nation number': nan,
   'nation': nan,
   'z': nan},),
 'SArea': ({'y': 31,
   'rainfall': 19.0,
   'x': 14,
   'key': '14:31',
   'elevation': 2.0,
   'terrain': 'land',
   'feature': nan,
   'nation number': 0.0,
   'nation': 'Flamehaven',
   'z': 'Flamehaven'},),
 'EArea': ({'y': 30,
   'rainfall': 9.0,
   'x': 15,
   'key': '15:30',
   'elevation': -1.0,
   'terrain': 'ocean',
   'feature': nan,
   'nation number': nan,
   'nation': nan,
   'z': nan},),
 'WArea': ({'y': 30,
   'rainfall': 18.0,
   'x': 13,
   'key': '13:30',

In [12]:
builders.world.chordKey([1,2])

'1:2'

In [7]:
":".join([1,2])

TypeError: sequence item 0: expected str instance, int found